# Analyse Results

This notebook contains the detailed analysis for all experiments for this bachelor project. It queries the evaluation data from W&B and displays/ visualises it nicely in `pd.DataFrames` (that are later compiled into LaTeX tables) and figures that are saved to the directory `reports/figures`.

In [ ]:
import wandb
import torch
import pandas as pd

# plotting
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
WANDB_PROJECT = "mikasenghaas/bsc"

In [ ]:
# get all runs
api = wandb.Api()
all_runs = api.runs(WANDB_PROJECT)

In [ ]:
# helper
def render_latex(df):
    # capitalise col names
    df.columns = [' '.join(map(lambda x: x[0].upper() + x[1:], col.split('_'))) for col in df.columns]
    
    # format df
    s = df.style.highlight_max(props='bfseries: ;')
    s.format(precision=2)
    
    # render latex
    opts = {"hrules": True, "position": "h"}
    return s.to_latex(**opts)

## Experiment 1: Image Classifiers

This experiment compares a wide-variety of commonly used image classifies. The classifiers tested are freely available on the PyTorch Hub. The following tables shows an overview over all models

In [ ]:
runs = [run for run in all_runs if run.group == "experiment1"]

print(f"There are {len(runs)} runs in Experiment 1 ({', '.join([run.name for run in runs])})")

### Configuration

In [ ]:
# convert wandb config to df
config = pd.DataFrame([dict(run.config, **{"id": run.name}) for run in runs])

# drop unnecessary cols
config = config.drop(["model", "version", "wandb_name", "wandb_log", "wandb_tags", "all_classes", "first_floor", "ground_floor", "wandb_group", "include_classes"], axis=1)

# set index
config = config.set_index("id")
config

In [ ]:
# render latex
training_params = ["max_epochs", "batch_size", "lr", "gamma", "step_size", "device"]
data_params = ["ratio"]

print(render_latex(config[training_params]))

### Results

In [ ]:
# convert wandb summary to df
summary = pd.DataFrame([dict({k: v for k, v in run.summary.items() if not k.startswith("_")}, **{"id": run.name}) for run in runs])

# drop unnecessary cols
summary = summary.drop([], axis=1)

# set index
summary = summary.set_index("id")
summary

In [ ]:
# render latex
print(render_latex(summary))

## Experiment 2

This experiment compares different kinds of video classifiers against each other.

In [ ]:
runs = [run for run in all_runs if run.group == "experiment2"]

print(f"There are {len(runs)} runs in Experiment 2 ({', '.join([run.name for run in runs])})")

### Configuration

In [ ]:
# convert wandb config to df
config = pd.DataFrame([dict(run.config, **{"id": run.name}) for run in runs])

# drop unnecessary cols
config = config.drop(["model", "version", "wandb_name", "wandb_log", "wandb_tags", "all_classes", "first_floor", "ground_floor", "wandb_group", "include_classes"], axis=1)

# set index
config = config.set_index("id")
config

### Results

In [ ]:
# convert wandb summary to df
summary = pd.DataFrame([dict({k: v for k, v in run.summary.items() if not k.startswith("_")}, **{"id": run.name}) for run in runs])

# drop unnecessary cols
summary = summary.drop([], axis=1)

# set index
summary = summary.set_index("id")
summary

In [ ]:
# render latex
print(render_latex(summary))